# Dawid-Skene法によるクラウドソーシング結果集約
<a target="_blank" href="https://colab.research.google.com/github/takumi1001/seccamp2024_D2/blob/main/02_Dawid-Skene_Aggregation">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## 準備

### Google Colabにインストールされていないライブラリを導入する

In [ ]:
!pip install crowd-kit

### ライブラリをインポートする

In [1]:
import numpy as np
import pandas as pd

クラウドソーシング企業Toloka社の公開しているライブラリ`crowd-kit`を使うと，様々な集約アルゴリズムやデータセットなどを簡単に利用できる．

https://github.com/Toloka/crowd-kit

In [2]:
from crowdkit.datasets.load_dataset import load_dataset
from crowdkit.aggregation.classification import DawidSkene, MajorityVote

### データセットをダウンロードする
ワーカ間一致率と同じように`relevance-2`データセットを用いる．

今回は，集約方法によって品質が変わることを確認したいので，
`relevance-2`のデータのうち正解データがついているものだけを利用する．

In [3]:
df, gt = load_dataset('relevance-2')

# 正解データがついているタスクのみを抽出する
df = df.merge(gt, on="task", how="left", validate="m:1")
df = df[df["true_label"].notnull()].drop(columns=["true_label"])

データを見てみましょう

In [4]:
df

,worker,task,label
0,w851,t30685,1
8,w5879,t7116,1
14,w3048,t95532,0
25,w3716,t12806,1
45,w5843,t90925,0
...,...,...,...
475516,w4807,t32606,0
475518,w1021,t45301,1
475521,w4311,t83768,1
475522,w6785,t21709,0


正解データのことをGround Truth，略してgtということが良くあります．

gtデータの中身も確認してみましょう．

In [5]:
gt

task
t30006    0
t33578    0
t22462    1
t52093    0
t26935    0
         ..
t57345    1
t81052    1
t7189     1
t80463    0
t93643    0
Name: true_label, Length: 10079, dtype: int64

## 単純多数決による集約
単純多数決を用いて，結果を集約し，品質を確認してみましょう．

In [6]:
# crowd-kitの集約アルゴリズムは，(worker,task,label)のDataFrameを入力として渡すだけで使える
majority_vote = MajorityVote() # インスタンス化する必要がある
mv_result = majority_vote.fit_predict(df)

In [7]:
mv_result.head() # 結果データを覗いてみる

task
t10001    1
t1003     1
t10031    0
t10037    1
t10040    1
Name: agg_label, dtype: int64

### 精度を計測する
精度として accuracy (正解率) を計測してみましょう．

In [8]:
from sklearn.metrics import accuracy_score # ライブラリからインポートする

In [9]:
# 精度を計測するために，gtデータからy_true配列を作成する
y_true = gt.sort_index().to_numpy() 

In [10]:
y_true

array([0, 0, 1, ..., 0, 0, 0], dtype=int64)

In [11]:
# 同じく，mv_resultからy_pred配列を作成する
y_pred = mv_result.sort_index().to_numpy()

In [12]:
y_pred

array([1, 1, 0, ..., 1, 1, 0], dtype=int64)

In [13]:
accuracy_mv = accuracy_score(y_true, y_pred)

In [14]:
accuracy_mv

0.7763667030459371

単純多数決の精度は`77.6%`でした．

## Dawid-Skene法での集約
続いて，Dawid-Skene法を用いてみましょう．

In [15]:
# crowd-kitの集約アルゴリズムは，(worker,task,label)のDataFrameを入力として渡すだけで使える
dawid_skene = DawidSkene() # インスタンス化する必要がある
ds_result = dawid_skene.fit_predict(df)

In [16]:
ds_result.head()

task
t30685    1
t7116     1
t95532    1
t12806    1
t90925    1
Name: agg_label, dtype: int64

単純多数決と同じように，精度を計算します．

In [21]:
y_true = gt.sort_index().to_numpy() 
y_pred = ds_result.sort_index().to_numpy()
accuracy_ds = accuracy_score(y_true, y_pred)

In [22]:
accuracy_ds

0.794820914773291

In [23]:
accuracy_mv

0.7763667030459371

単純多数決と比べて，Dawid-Skene法の方が精度が高いことがわかります．

ただし，常に`Dawid-Skene法>単純多数決`が成り立つわけではありません，データによって最適なアルゴリズムは異なる点には注意が必要です．クラウドソーシングではワーカ間の能力差が幅広い傾向がありますので，一般にはDawid-Skene法が有効であると考えられています．

## スパムワーカを追加する
ワーカ間一致率の場合と同じように，悪いふるまいをするワーカを追加して，集約結果がどう変化するか，試してみましょう．

悪いふるまいをするワーカはすべてのタスクにランダムに回答します

In [24]:
num_spam_workers = 20 # スパムワーカ数

In [25]:
# ランダムな振る舞いをするワーカを生成する
rows = []
for i_worker in range(num_spam_workers):
    worker_id = f"random_{i_worker}"
    for task_id in df["task"].unique():
        row = (worker_id, task_id, np.random.randint(low=0,high=1+1))
        rows.append(row)
spam_df = pd.DataFrame(rows, columns=["worker","task","label"])

In [31]:
spam_df.head()

,worker,task,label
0,random_0,t30685,0
1,random_0,t7116,1
2,random_0,t95532,0
3,random_0,t12806,1
4,random_0,t90925,1


In [27]:
df_with_spam = pd.concat([df, spam_df]) # データセットと結合する

### 単純多数決

In [28]:
majority_vote = MajorityVote() # インスタンス化する必要がある
mv_result_with_spam = majority_vote.fit_predict(df_with_spam)
y_true = gt.sort_index().to_numpy() 
y_pred = mv_result_with_spam.sort_index().to_numpy()
accuracy_score(y_true, y_pred)

0.6498660581406885

### Dawid-Skene法

In [29]:
dawid_skene = DawidSkene() # インスタンス化する必要がある
ds_result_with_spam = dawid_skene.fit_predict(df_with_spam)
y_true = gt.sort_index().to_numpy() 
y_pred = ds_result_with_spam.sort_index().to_numpy()
accuracy_score(y_true, y_pred)

0.7918444290108145

Dawid-Skene法が品質の悪いワーカの影響をほとんど受けていなことがわかると思います．

## 余裕があればやってみよう
 - スパムワーカの数を増減して，Dawid-Skene法や単純多数決の精度がどう変化するか，試してみよう
 - Dawid-Skene法以外の集約アルゴリズムを試してみよう．
   - crowd-kitのドキュメントを参考にしよう：https://crowd-kit.readthedocs.io/en/stable/classification/
   - `MACE`や`GLAD`，`OneCoinDawidSkene`などは試しやすくておすすめ
   - これらの手法とDawid-Skene法の違いはなんだろうか？
 - Dawid-Skene法のパラメータを変更していろいろ試してみよう
   - EMアルゴリズムの繰り返し回数はどのように制御されている？
 - `relevance-2`以外のデータセットに適用してみよう．
   - 集約アルゴリズムはクラス分類問題にしか適用できないので注意しよう
   - オープンデータについては以下のサイトにまとまっている
   - https://github.com/TrentoCrowdAI/crowdsourced-datasets
   - 自分でデータをダウンロードし，変換しないといけないので少し大変